# DSCI 591 Data Analysis
## Covid-19

In [8]:
import os
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql import SparkSession
import findspark

from ipynb.fs.full.covid_data_collection import get_counties_df

### Initialize Spark

In [2]:
findspark.init() 

In [3]:
spark = SparkSession.builder.getOrCreate()

### Merge and reformat COVID data by date and FIPS

In [46]:
def get_fips_from_county_lookup():
    counties = get_counties_df() 
    counties = counties.dropna(subset=['fips'])
    counties.fips = counties.fips.apply(lambda f: eval(str(f)).get('id'))
    counties_index = {k: v for k, v in zip(counties.index, counties.fips)}
    def get_fips_from_county(county):
        return counties_index[county]
    return get_fips_from_county
                                  
get_fips_from_country = get_fips_from_county_lookup()

In [47]:
get_fips_from_country('Baldwin_Alabama_UnitedStates')


'01003'

In [6]:
df = pd.read_pickle('./processed_data/county_merged_parts/Niagara_NewYork_UnitedStates.pkl')
sdf = ps.DataFrame(df)


In [7]:
sdf.head()

,JHU_ConfirmedCases.data,JHU_ConfirmedCases.missing,NYT_ConfirmedCases.data,NYT_ConfirmedCases.missing,JHU_ConfirmedDeaths.data,JHU_ConfirmedDeaths.missing,JHU_ConfirmedRecoveries.data,JHU_ConfirmedRecoveries.missing,NYT_AllCausesDeathsWeekly_Expected_Deaths_AllCauses.data,NYT_AllCausesDeathsWeekly_Expected_Deaths_AllCauses.missing,NYT_ConfirmedDeaths.data,NYT_ConfirmedDeaths.missing,NYT_AllCausesDeathsWeekly_Excess_Deaths.data,NYT_AllCausesDeathsWeekly_Excess_Deaths.missing,NYT_AllCausesDeathsWeekly_Deaths_AllCauses.data,NYT_AllCausesDeathsWeekly_Deaths_AllCauses.missing,NYT_AllCausesDeathsMonthly_Deaths_AllCauses.data,NYT_AllCausesDeathsMonthly_Deaths_AllCauses.missing,NYT_AllCausesDeathsMonthly_Excess_Deaths.data,NYT_AllCausesDeathsMonthly_Excess_Deaths.missing,NYT_AllCausesDeathsMonthly_Expected_Deaths_AllCauses.data,NYT_AllCausesDeathsMonthly_Expected_Deaths_AllCauses.missing,TotalPopulation.data,TotalPopulation.missing,MaleAndFemale_AtLeast65_Population.data,MaleAndFemale_AtLeast65_Population.missing,Male_Total_Population.data,Male_Total_Population.missing,Female_Total_Population.data,Female_Total_Population.missing,MaleAndFemale_Under18_Population.data,MaleAndFemale_Under18_Population.missing,BLS_EmployedPopulation.data,BLS_EmployedPopulation.missing,BLS_UnemployedPopulation.data,BLS_UnemployedPopulation.missing,BLS_UnemploymentRate.data,BLS_UnemploymentRate.missing,BLS_LaborForcePopulation.data,BLS_LaborForcePopulation.missing,AverageDailyTemperature.data,AverageDailyTemperature.missing,AverageDewPoint.data,AverageDewPoint.missing,AverageRelativeHumidity.data,AverageRelativeHumidity.missing,AverageSurfaceAirPressure.data,AverageSurfaceAirPressure.missing,AveragePrecipitationTotal.data,AveragePrecipitationTotal.missing,AveragePrecipitation.data,AveragePrecipitation.missing,AverageWindDirection.data,AverageWindDirection.missing,AverageWindSpeed.data,AverageWindSpeed.missing,hospitalIcuBeds,hospitalStaffedBeds,hospitalLicensedBeds,latestTotalPopulation,fips,LND110210
dates,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,0.0,0,0.0,100,0.0,0,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,209281.0,0,40109.0,0,102493.0,0,107940.0,0,42064.0,0,93664.5,0,5921.0,0,5.945645,0,99585.5,0,29.708333,0,23.194444,0,76.750000,0,29.087639,0,0.004167,0,0.000000,0,247.638889,0,15.013889,0,32.0,411.0,482.0,209281.0,36063.0,522.36
2020-01-02,0.0,0,0.0,100,0.0,0,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,209281.0,0,40109.0,0,102493.0,0,107940.0,0,42064.0,0,93664.5,0,5921.0,0,5.945645,0,99585.5,0,40.375000,0,29.208333,0,65.125000,0,29.062917,0,0.000000,0,0.000000,0,214.583333,0,17.354167,0,32.0,411.0,482.0,209281.0,36063.0,522.36
2020-01-03,0.0,0,0.0,100,0.0,0,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,209281.0,0,40109.0,0,102493.0,0,107940.0,0,42064.0,0,93664.5,0,5921.0,0,5.945645,0,99585.5,0,42.625000,0,37.638889,0,82.500000,0,29.110764,0,0.000000,0,0.000000,0,221.250000,0,8.243056,0,32.0,411.0,482.0,209281.0,36063.0,522.36
2020-01-04,0.0,0,0.0,100,0.0,0,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,209281.0,0,40109.0,0,102493.0,0,107940.0,0,42064.0,0,93664.5,0,5921.0,0,5.945645,0,99585.5,0,33.195833,0,29.506944,0,86.591667,0,29.127806,0,0.000667,0,0.003958,0,208.250000,0,9.337500,0,32.0,411.0,482.0,209281.0,36063.0,522.36
2020-01-05,0.0,0,0.0,100,0.0,0,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,0.0,100,209281.0,0,40109.0,0,102493.0,0,107940.0,0,42064.0,0,93664.5,0,5921.0,0,5.945645,0,99585.5,0,31.388889,0,25.612500,0,79.532341,0,29.287247,0,0.001667,0,0.001729,0,255.400794,0,11.925893,0,32.0,411.0,482.0,209281.0,36063.0,522.36


In [9]:
filename = '../air_quality/no2/daily_2020_multi_index.pkl'
df = pd.read_pickle(filename)
daily_no2 = ps.DataFrame(df)

In [10]:
daily_no2.head()

Arithmetic Mean  1st Max Value  AQI
Date Local fips                                      
2020-01-01 01073        16.373918           31.0   29
           04013        16.660833           34.0   32
           04019        11.206250           23.7   22
           05035         9.175000           23.1   22
           05119        12.637500           28.9   26

In [11]:
df.reset_index()

,Date Local,fips,Arithmetic Mean,1st Max Value,AQI
0,2020-01-01,01073,16.373918,31.0,29
1,2020-01-01,04013,16.660833,34.0,32
2,2020-01-01,04019,11.206250,23.7,22
3,2020-01-01,05035,9.175000,23.1,22
4,2020-01-01,05119,12.637500,28.9,26
...,...,...,...,...,...
90835,2020-12-31,56021,4.660000,14.6,13
90836,2020-12-31,56025,6.164167,30.6,28
90837,2020-12-31,56035,1.275326,7.5,7
90838,2020-12-31,56037,4.892572,28.4,26


In [ ]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df